## Binary classification : Cats and Dogs

In [ ]:
### this is the binary classification for the cats and dog images
### here I am just gonna preprocess the raw images and then
### use the CNN model to make the predictions
### later on in other file, I will do the reverse engineering to check for more details

### loading the libraries

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img, load_img

### loading and preprocessing the images : Train

In [2]:
basedir = '../data/raw/train'
target_size = (224, 224)
X, y = [], []
class_labels = {'cats' : 0, 'dogs' : 1}

In [4]:
for class_name in os.listdir(basedir) : 
    class_path = os.path.join(basedir, class_name)

    if not os.listdir(basedir) : 
        continue

    print(f"loading the images : {class_name}")

    for class_images in os.listdir(class_path) : 
        image_path = os.path.join(class_path, class_images)

        try : 
            load_images = load_img(image_path, target_size = target_size)
            load_images = img_to_array(load_images) / 255.0

            X.append(load_images)
            y.append(class_labels[class_name])

        except Exception as e : 
            print(f"error loading the image : {image_path}, {e}")

loading the images : cats
loading the images : dogs


In [7]:
X = np.array(X)
y = np.array(y)

In [8]:
print(f"{X.shape} : {y.shape}")

(557, 224, 224, 3) : (557,)


In [9]:
X = np.save('../data/preprocessed/X_train.npy', X)
y = np.save('../data/preprocessed/y_train.npy', y)

### loading and preprocessing the images : Test